In [1]:
# ======================================
#  Notebook: Image Recommendation System
# ======================================

# 1. Setup do ambiente
!pip install keras tensorflow opencv-python matplotlib scikit-learn

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.neighbors import NearestNeighbors


In [2]:
# 2. Carregar modelo pré-treinado (sem a última camada)
model = ResNet50(weights="imagenet", include_top=False, pooling="avg")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
# 3. Função para extrair features
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()

In [ ]:
# 4. Criar dataset de embeddings
image_dir = "imagens/"  # coloque suas imagens aqui
filenames = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".jpg")]

feature_list = []
for file in filenames:
    feature_list.append(extract_features(file, model))

feature_list = np.array(feature_list)

In [ ]:
# 5. Treinar KNN para recomendação
neighbors = NearestNeighbors(n_neighbors=5, metric="cosine")
neighbors.fit(feature_list)

In [ ]:
# 6. Função de recomendação
def recommend(img_path):
    features = extract_features(img_path, model)
    distances, indices = neighbors.kneighbors([features])
    plt.figure(figsize=(15,5))
    plt.subplot(1, 6, 1)
    plt.imshow(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))
    plt.title("Input")
    plt.axis("off")
    for i, idx in enumerate(indices[0]):
        plt.subplot(1, 6, i+2)
        plt.imshow(cv2.cvtColor(cv2.imread(filenames[idx]), cv2.COLOR_BGR2RGB))
        plt.title(f"Sim {i+1}")
        plt.axis("off")
    plt.show()

In [ ]:
# 7. Testar recomendação
recommend("imagens/teste.jpg")